In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Sample data
df = px.data.gapminder().query("year == 2007")

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Simple Dashboard"),
    dcc.Graph(
        figure=px.scatter(df, x="gdpPercap", y="lifeExp", size="pop", color="continent", hover_name="country")
    )
])

if __name__ == '__main__':
    app.run(debug=True)  # Replaced app.run_server() with app.run()
